In [9]:
import pandas as pd
import numpy as np
import csv

In [10]:
glass_data = pd.read_csv(r'E:\Coding\Data science practice\kaggle_a\glassdoor_scraper\pay.csv', 
                         header=None)
glass_data.columns = ['Job', 'country', 'Currency', 'Avg_pay', 'Frequency', 'Confidence']
glass_data = glass_data[glass_data['country'] != 'Viet Nam']   # this columns was a mistake during defining country 
                                                               #names. Original Vietnam is still in the axis.
glass_data['country'].unique()

array(['Switzerland', 'USA', 'Israel', 'Australia', 'UK', 'Canada',
       'Germany', 'Netherlands', 'UAE', 'Sweden', 'Belgium', 'Singapore',
       'Ireland', 'France', 'Japan', 'Spain', 'South Africa', 'Italy',
       'South Korea', 'Republic of Korea', 'China', 'Saudi Arabia',
       'Poland', 'Portugal', 'Mexico', 'Thailand', 'Romania', 'Taiwan',
       'Chile', 'Greece', 'Brazil', 'Malaysia', 'Russia', 'India',
       'Ukraine', 'Argentina', 'Tunisia', 'Turkey', 'Philippines', 'Peru',
       'Colombia', 'Belarus', 'Indonesia', 'Morocco', 'Pakistan', 'Egypt',
       'Sri Lanka', 'Nigeria', 'Kenya', 'Iran', 'Bangladesh', 'Vietnam'],
      dtype=object)

In [11]:
conversion_rates = pd.read_csv(r'E:\Coding\Data science practice\kaggle_a\glassdoor_scraper\cleaning\conversion_rate_16dec2020.csv', 
                         header=0)
# source: https://www.bis.org/statistics/full_webstats_xru_current_d_dataflow_csv_col.zip

conv_df = conversion_rates[['Reference area', 'CURRENCY', 'Currency', '2020-12-14']].copy()
conv_df.columns = ['Reference area', 'CURRENCY', 'Currency', 'conversion(1 USD=)']
conv_df['Reference area'].replace({'United Arab Emirates': 'UAE', 'United Kingdom': 'UK', 'United States': 'USA'}, 
                                  inplace=True)
conv_df['Reference area'].unique()

array(['UAE', 'Albania', 'Argentina', 'Austria', 'Australia',
       'Bosnia and Herzegovina', 'Belgium', 'Bulgaria', 'Bahrain',
       'Brunei', 'Brazil', 'Canada', 'Switzerland', 'Chile', 'China',
       'Colombia', 'Cyprus', 'Czech Republic', 'Germany', 'Denmark',
       'Algeria', 'Estonia', 'Spain', 'Finland', 'France', 'UK', 'Greece',
       'Hong Kong SAR', 'Croatia', 'Hungary', 'Indonesia', 'Ireland',
       'Israel', 'India', 'Iran', 'Iceland', 'Italy', 'Japan',
       'South Korea', 'Kuwait', 'Kazakhstan', 'Sri Lanka', 'Lithuania',
       'Luxembourg', 'Latvia', 'North Macedonia', 'Malta', 'Mauritius',
       'Mexico', 'Malaysia', 'Netherlands', 'Norway', 'Nepal',
       'New Zealand', 'Oman', 'Peru', 'Philippines', 'Pakistan', 'Poland',
       'Portugal', 'Qatar', 'Romania', 'Serbia', 'Russia', 'Saudi Arabia',
       'Sweden', 'Singapore', 'Slovenia', 'Slovakia', 'Thailand',
       'Tunisia', 'Turkey', 'Trinidad and Tobago', 'Chinese Taipei',
       'Ukraine', 'USA', 'Urugua

In [12]:
tofilldata = [['Republic of Korea',None, None, 1108.76], ['Taiwan',None,None,28.18], ['Belarus',None, None,  2.50],
              ['Morocco',None,None,8.96], ['Egypt',None,None,15.68], ['Nigeria',None,None,380.00], 
              ['Kenya',None, None, 111.09], ['Bangladesh',None,None,84.31], ['Vietnam',None,None,23134.50]]
filler =pd.DataFrame(tofilldata, columns = ['Reference area', 'CURRENCY', 'Currency', 'conversion(1 USD=)'])
conv_df = pd.concat([conv_df, filler])
conv_df.tail()

,Reference area,CURRENCY,Currency,conversion(1 USD=)
4,Egypt,None,None,15.68
5,Nigeria,None,None,380.00
6,Kenya,None,None,111.09
7,Bangladesh,None,None,84.31
8,Vietnam,None,None,23134.50


In [13]:
glass_df = glass_data.merge(conv_df, how='left', left_on='country', right_on='Reference area')

In [14]:
null_data = glass_df[glass_df['Reference area'].isnull()]
null_countries = null_data['country'].unique() # we need to manually update the rates if possible.
null_countries 

array([], dtype=object)

In [15]:
glass_df['Avg_pay(USD)'] = glass_df['Avg_pay']/glass_df['conversion(1 USD=)']
glass_df[glass_df['country']=='India']
finalpay_df = glass_df[['Job','country', 'Currency_x', 'Currency_y', 'CURRENCY','Avg_pay','conversion(1 USD=)',
                         'Avg_pay(USD)', 'Frequency', 'Confidence']]

In [16]:
finalpay_df.to_csv('finalpay.csv', index=False)